In [6]:
from PyCoalescedTsetlinMachineCUDA.tm import MultiClassConvolutionalTsetlinMachine2D
import numpy as np
from time import time
from keras.datasets import cifar10
import cv2

In [7]:
clauses = 40000
T = 7500
s = 20.0
patch_size = 8
resolution = 8
epochs = 50

In [8]:
def horizontal_flip(image):
    return cv2.flip(image, 1)

augmented_images = []
augmented_labels = []

labels = [b'airplane', b'automobile', b'bird', b'cat', b'deer', b'dog', b'frog', b'horse', b'ship', b'truck']

(X_train_org, Y_train), (X_test_org, Y_test) = cifar10.load_data()

for i in range(len(X_train_org)):
    image = X_train_org[i]
    label = Y_train[i]

    # Original image and label
    augmented_images.append(image)
    augmented_labels.append(label)

augmented_images.append(horizontal_flip(image))
augmented_labels.append(label)

X_train_aug = np.array(augmented_images)
Y_train = np.array(augmented_labels).reshape(-1, 1)

X_train = np.copy(X_train_aug)
X_test = np.copy(X_test_org)

Y_train = Y_train.reshape(Y_train.shape[0])
Y_test = Y_test.reshape(Y_test.shape[0])

In [9]:
for i in range(X_train.shape[0]):
    for j in range(X_train.shape[3]):
        X_train[i, :, :, j] = cv2.adaptiveThreshold(
            X_train_aug[i, :, :, j],
            1,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            11,
            2,
        )

for i in range(X_test.shape[0]):
    for j in range(X_test.shape[3]):
        X_test[i, :, :, j] = cv2.adaptiveThreshold(
            X_test_org[i, :, :, j],
            1,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            11,
            2,
        )

In [11]:
f = open("cifar10_%.1f_%d_%d_%d.txt" % (s, clauses, T,  patch_size), "w+")


tm = MultiClassConvolutionalTsetlinMachine2D(clauses, T, s, (patch_size, patch_size), q=1.0, number_of_state_bits=8)

for i in range(epochs):
        start_training = time()
        tm.fit(X_train, Y_train, epochs=1, incremental=True)
        stop_training = time()

        start_testing = time()
        result_test = 100*(tm.predict(X_test) == Y_test).mean()
        stop_testing = time()

        result_train = 100*(tm.predict(X_train) == Y_train).mean()
        print("%d %.2f %.2f %.2f %.2f" % (i, result_train, result_test, stop_training-start_training, stop_testing-start_testing))
        print("%d %.2f %.2f %.2f %.2f" % (i, result_train, result_test, stop_training-start_training, stop_testing-start_testing), file=f)
        f.flush()
    
f.close()

0 55.24 52.67 113.45 14.46
1 62.49 57.44 110.16 14.37
2 67.21 59.88 110.01 14.32
3 70.85 61.44 108.94 14.05
4 73.70 61.58 106.66 13.95
5 76.10 62.29 106.11 13.76
6 78.11 62.79 104.88 14.03
7 80.26 63.35 105.14 13.52
8 81.76 63.69 104.62 14.04
9 82.93 64.48 105.48 14.14
10 84.40 63.62 104.92 14.11
11 85.50 64.17 105.40 14.44
12 86.45 64.35 106.47 14.49
13 86.97 64.34 105.69 14.07
14 87.98 63.73 106.02 14.32
15 88.73 63.66 107.34 14.46
16 89.23 63.81 107.53 14.13
17 89.94 63.79 107.49 14.26
18 90.61 63.73 107.81 14.07
19 90.70 63.64 107.69 14.48
20 91.18 63.70 106.89 14.10
21 91.55 63.33 106.74 14.14
22 91.86 63.59 106.96 14.35
23 92.23 63.89 106.41 13.81
24 92.54 64.09 105.85 14.19
25 92.61 63.43 105.53 14.11
26 92.92 63.45 105.30 14.12
27 93.17 63.07 105.08 13.79
28 93.11 63.02 104.09 14.03
29 93.45 63.01 103.48 13.94
30 93.54 63.70 103.97 14.15
31 93.53 63.02 103.71 14.57
32 93.68 62.45 103.80 14.16
33 93.77 62.64 104.43 14.05
34 93.82 62.58 104.85 14.14
35 94.22 62.99 103.72 13.90
36